In [12]:
# The following is taken from 
# https://www.thepythoncode.com/article/get-hardware-system-information-python#System_Information

import psutil
import sys
import platform
from datetime import datetime

In [13]:
def get_size(bytes, suffix="B"):
    '''
    Scale bytes to its proper format 
    e.g:
        1253656 => '1.2MB'S
    '''
    factor = 1024
    for unit in ['','K','M','G','T','P']:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

In [14]:
## System information ##################################################
uname = platform.uname()
print(f"System: {uname.system}")
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

System: Linux
System: Linux
Node Name: box
Version: #60-Ubuntu SMP Fri Nov 6 10:37:59 UTC 2020
Machine: x86_64
Processor: x86_64


In [15]:
## Date and time PC was booted #########################################
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

Boot Time: 2020/11/22 21:44:44


In [16]:
## CPU Information #####################################################

# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))

# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max: .2f}Mhz")
print(f"Min Frequency: {cpufreq.min: .2f}Mhz")
print(f"Current Frequency: {cpufreq.current: .2f}Mhz")

# CPU usage 
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

Physical cores: 8
Total cores: 16
Max Frequency:  5300.00Mhz
Min Frequency:  800.00Mhz
Current Frequency:  3400.84Mhz
CPU Usage Per Core:
Core 0: 11.9%
Core 1: 14.9%
Core 2: 17.6%
Core 3: 16.5%
Core 4: 16.7%
Core 5: 10.1%
Core 6: 26.2%
Core 7: 22.1%
Core 8: 22.6%
Core 9: 35.5%
Core 10: 17.9%
Core 11: 10.1%
Core 12: 32.4%
Core 13: 23.3%
Core 14: 8.8%
Core 15: 7.1%
Total CPU Usage: 20.4%


In [17]:
## Memory Information #################################################

# Get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")

# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")

Total: 31.00GB
Available: 8.47GB
Used: 21.55GB
Percentage: 72.7%
Total: 2.00GB
Free: 1.87GB
Used: 129.71MB


In [18]:
## Disk Usage ####################################################

# Disk Information
print("Partitions and Usage:")

# Get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f" Mountpoint: {partition.mountpoint}")
    print(f" File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that isn't ready
        continue
    print(f"   Total Size: {get_size(partition_usage.total)}")
    print(f"   Used: {get_size(partition_usage.used)}")
    print(f"   Free: {get_size(partition_usage.free)}")
    print(f"   Percentage: {partition_usage.percent}%")

# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

Partitions and Usage:
=== Device: /dev/nvme0n1p2 ===
 Mountpoint: /
 File system type: ext4
   Total Size: 937.40GB
   Used: 89.42GB
   Free: 800.30GB
   Percentage: 10.1%
=== Device: /dev/loop1 ===
 Mountpoint: /snap/code/50
 File system type: squashfs
   Total Size: 146.12MB
   Used: 146.12MB
   Free: 0.00B
   Percentage: 100.0%
=== Device: /dev/loop0 ===
 Mountpoint: /snap/code/49
 File system type: squashfs
   Total Size: 142.38MB
   Used: 142.38MB
   Free: 0.00B
   Percentage: 100.0%
=== Device: /dev/loop2 ===
 Mountpoint: /snap/core/10185
 File system type: squashfs
   Total Size: 97.75MB
   Used: 97.75MB
   Free: 0.00B
   Percentage: 100.0%
=== Device: /dev/loop4 ===
 Mountpoint: /snap/spotify/43
 File system type: squashfs
   Total Size: 173.00MB
   Used: 173.00MB
   Free: 0.00B
   Percentage: 100.0%
=== Device: /dev/loop3 ===
 Mountpoint: /snap/snap-store/481
 File system type: squashfs
   Total Size: 50.75MB
   Used: 50.75MB
   Free: 0.00B
   Percentage: 100.0%
=== Device: /d

In [19]:
## Network Information ###############################################
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")
        

=== Interface: lo ===
  IP Address: 127.0.0.1
  Netmask: 255.0.0.0
  Broadcast IP: None
=== Interface: lo ===
=== Interface: lo ===
  MAC Address: 00:00:00:00:00:00
  Netmask: None
  Broadcast MAC: None
=== Interface: wlp0s20f3 ===
  IP Address: 10.0.0.169
  Netmask: 255.255.255.0
  Broadcast IP: 10.0.0.255
=== Interface: wlp0s20f3 ===
=== Interface: wlp0s20f3 ===
=== Interface: wlp0s20f3 ===
=== Interface: wlp0s20f3 ===
=== Interface: wlp0s20f3 ===
  MAC Address: 34:2e:b7:0d:c8:cf
  Netmask: None
  Broadcast MAC: ff:ff:ff:ff:ff:ff
=== Interface: virbr0 ===
  IP Address: 192.168.122.1
  Netmask: 255.255.255.0
  Broadcast IP: 192.168.122.255
=== Interface: virbr0 ===
  MAC Address: 52:54:00:33:71:fd
  Netmask: None
  Broadcast MAC: ff:ff:ff:ff:ff:ff
=== Interface: vnet0 ===
=== Interface: vnet0 ===
  MAC Address: fe:54:00:b6:3f:1f
  Netmask: None
  Broadcast MAC: ff:ff:ff:ff:ff:ff
=== Interface: virbr0-nic ===
  MAC Address: 52:54:00:33:71:fd
  Netmask: None
  Broadcast MAC: ff:ff:ff:ff

In [20]:
## Get Temperature Sensor Data

if not hasattr(psutil, "sensors_temperatures"):
    sys.exit("platform not supported")
temps = psutil.sensors_temperatures()
if not temps:
    sys.exit("can't read any temperature")
for name, entries in temps.items():
    print(name)
    for entry in entries:
        print("    %-20s %s °C (high = %s °C, critical = %s °C)" % (
            entry.label or name, entry.current, entry.high,
            entry.critical))
    print()

acpitz
    acpitz               67.0 °C (high = 128.0 °C, critical = 128.0 °C)

pch_cometlake
    pch_cometlake        75.0 °C (high = None °C, critical = None °C)

thinkpad
    thinkpad             67.0 °C (high = None °C, critical = None °C)
    thinkpad             0.0 °C (high = None °C, critical = None °C)
    thinkpad             1.0 °C (high = None °C, critical = None °C)
    thinkpad             0.0 °C (high = None °C, critical = None °C)
    thinkpad             0.0 °C (high = None °C, critical = None °C)
    thinkpad             0.0 °C (high = None °C, critical = None °C)
    thinkpad             0.0 °C (high = None °C, critical = None °C)
    thinkpad             2.0 °C (high = None °C, critical = None °C)
    thinkpad             54.0 °C (high = None °C, critical = None °C)
    thinkpad             69.0 °C (high = None °C, critical = None °C)
    thinkpad             0.0 °C (high = None °C, critical = None °C)
    thinkpad             69.0 °C (high = None °C, critical = Non

In [21]:
## Get Fan Data ############################################################
if not hasattr(psutil, "sensors_fans"):
    sys.exit("platform not supported")  # add return statement in function
fans = psutil.sensors_fans()
if not fans:
    print("no fans detected")
for name, entries in fans.items():
    print(name)
    for entry in entries:
        print("    %-20s %s RPM" % (entry.label or name, entry.current))
    print()

thinkpad
    thinkpad             3610 RPM



In [22]:
## Get Battery Data ########################################################

# Function to convert seconds to hours

##
if not hasattr(psutil, "sensors_battery"):
    sys.exit("platform not supported")  # return
batt = psutil.sensors_battery()
if batt is None:
    sys.exit("no battery is installed")  # return
print("charge:     %s%%" % round(batt.percent, 2))
if batt.power_plugged:
    print("status:     %s" % (
        "charging" if batt.percent < 100 else "fully charged"))
    print("plugged in: yes")
else:
    print("left:       %s" % secs2hours(batt.secsleft))
    print("status:     %s" % "discharging")
    print("plugged in: no")

charge:     92.33%
status:     charging
plugged in: yes


In [23]:
## Get users connected to the system #######################################
psutil.users()

[suser(name='nchannelfet', terminal=':1', host=':1', started=1606110336.0, pid=1701)]

In [10]:
## Get GPU Info ############################################################
import GPUtil

gpus = GPUtil.getGPUs()
list_gpus = []
for gpu in gpus:
    # get the GPU id
    gpu_id = gpu.id
    # name of GPU
    gpu_name = gpu.name
    # get % percentage of GPU usage of that GPU
    gpu_load = f"{gpu.load*100}%"
    # get free memory in MB format
    gpu_free_memory = f"{gpu.memoryFree}MB"
    # get used memory
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    # get total memory
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    # get GPU temperature in Celsius
    gpu_temperature = f"{gpu.temperature} °C"
    gpu_uuid = gpu.uuid
    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory, gpu_temperature, gpu_uuid
    ))

print(list_gpus)
print(f"\n GPU Name:      {gpu_name}")
print(f"\n GPU Load:      {gpu_load}")
print(f"\n GPU Free Mem:  {gpu_free_memory}")
print(f"\n GPU Used Mem:  {gpu_used_memory}")
print(f"\n GPU Total Mem: {gpu_total_memory}")
print(f"\n GPU Temp:      {gpu_temperature}")

[(0, 'GeForce GTX 1650 Ti with Max-Q Design', '6.0%', '3096.0MB', '815.0MB', '3911.0MB', '54.0 °C', 'GPU-7db19dfa-8697-1fdb-29b7-22de0171e3eb')]

 GPU Name:      GeForce GTX 1650 Ti with Max-Q Design

 GPU Load:      6.0%

 GPU Free Mem:  3096.0MB

 GPU Used Mem:  815.0MB

 GPU Total Mem: 3911.0MB

 GPU Temp:      54.0 °C


In [31]:
# GPU type
# :~$ lspci|grep -i nvidia
# If above command does not work
# :~$ sudo update-pciids

In [32]:
# Determine Ubuntu distro and release
# :~$ uname -m && cat /etc/*release

In [33]:
# Determine Linux Kernal Version
# :~$ uname -r